# 평균이동추적 Mean Shift Tracking

In [1]:
# image 작업은 numpy와 pyplot을 import 해야하고, vedio 작업은 ipywidgets과 Ipython을 import해야 합니다.
# ! pip install ipywidgets
import cv2
from ipywidgets import Image
from IPython.display import display

In [2]:
track_window = None  # 추적 객체의 위치 초기값
roi_hist = None  # 추적 객체의 히스토그램 초기값
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TermCriteria_COUNT, 10, 1)

In [3]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [4]:
ret, frame = cap.read()
x, y, w, h = cv2.selectROI("Select Object", frame, False, False)

In [5]:
roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

track_window = (x, y, w, h)

In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    
    ret, track_window = cv2.meanShift(dst, track_window, term_crit)
    
    x, y, w, h = track_window
    print(x, y, w, h)
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2. imshow("Mean Shift Tracking", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

286 203 102 73
286 203 102 73
287 203 102 73
287 203 102 73
287 203 102 73
281 203 102 73
283 205 102 73
284 205 102 73
284 206 102 73
285 208 102 73
285 209 102 73
288 209 102 73
288 209 102 73
285 211 102 73
288 211 102 73
290 211 102 73
289 211 102 73
286 210 102 73
286 209 102 73
287 209 102 73
287 208 102 73
286 207 102 73
290 210 102 73
290 209 102 73
294 211 102 73
296 211 102 73
301 214 102 73
303 214 102 73
303 214 102 73
301 214 102 73
303 217 102 73
303 217 102 73
303 217 102 73
304 216 102 73
304 216 102 73
304 216 102 73
306 214 102 73
305 213 102 73
304 212 102 73
301 209 102 73
301 213 102 73
301 213 102 73
311 218 102 73
318 221 102 73
318 223 102 73
316 223 102 73
311 234 102 73
314 224 102 73
313 224 102 73
312 223 102 73
312 220 102 73
308 219 102 73
318 226 102 73
320 234 102 73
320 234 102 73
322 248 102 73
325 264 102 73
330 273 102 73
330 273 102 73
331 273 102 73
330 271 102 73
330 272 102 73
331 272 102 73
331 273 102 73
329 273 102 73
329 273 102 73
331 273 10

538 272 102 73
538 272 102 73
538 275 102 73
538 277 102 73
538 277 102 73
538 278 102 73
538 281 102 73
538 281 102 73
538 281 102 73
538 284 102 73
538 285 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
538 287 102 73
508 287 102 73
466 287 102 73
456 287 102 73
437 287 102 73
437 287 102 73
430 287 102 73
441 285 102 73
442 286 102 73
448 287 102 73
448 287 102 73
464 287 102 73
462 287 102 73
462 287 102 73
465 287 102 73
465 287 102 73
469 287 102 73
477 287 102 73
461 287 102 73
443 287 102 73
443 287 102 73
442 287 102 73
441 287 102 73
456 287 102 73
462 287 102 73
458 287 102 73
458 287 102 73
459 287 102 73
449 283 102 73
451 283 102 73
448 279 102 73
443 277 102 73
448 277 102 73
450 276 102 73
451 276 102 73
451 277 102 73
452 277 102 73
463 278 102 73
465 280 102 73
468 282 102 73
471 282 10

# 칼만 필터 추적 Kalman Filter Tracking

In [7]:
import cv2
import numpy as np

In [8]:
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.05

In [9]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [10]:
ret, frame = cap.read()
bbox = cv2.selectROI("Select Object", frame, False, False)

In [11]:
kalman.statePre = np.array([[bbox[0]], [bbox[1]], [0], [0]], np.float32)

In [12]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    kalman.correct(np.array([[np.float32(bbox[0] + bbox[2] / 2)], [np.float32(bbox[1] + bbox[3] / 2)]]))
    kalman.predict()
    
    predicted_bbox = tuple(map(int, kalman.statePost[:2, 0]))
    
    cv2.rectangle(frame, (predicted_bbox[0] - bbox[2] // 2, predicted_bbox[1] - bbox[3] // 2), (predicted_bbox[0] + bbox[2] // 2, predicted_bbox[1] + bbox[3] // 2), (0, 255, 0), 2)
    
    cv2.imshow("Kalman Filter Tracking", frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 특징점 기반 추적 Feature-based Tracking

In [13]:
import cv2

In [14]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [15]:
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

In [16]:
lk_params = dict(sinSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [17]:
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

In [18]:
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
prev_points = prev_corners.squeeze()

In [19]:
color = (0, 255, 0)

초기화 단계

In [20]:
import cv2

In [21]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [22]:
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)  # Shi-Tomasi 코너 검출기 파라미터 설정

In [23]:
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [24]:
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

In [25]:
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
prev_points = prev_corners.squeeze()

In [26]:
color = (0, 255, 0)  # 특징점의 색깔은 초록색입니다.

In [27]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)
    
    for i, (prev_point, next_point) in enumerate(zip(prev_points, next_points)):
        x1, y1 = prev_point.astype(int)
        x2, y2 = next_point.astype(int)
        
        cv2.line(frame, (x1, y1), (x2, y2), color, 2)
        cv2.circle(frame, (x2, y2), 3, color, -1)
        
    cv2.imshow("Feature-based Tracking", frame)
    
    prev_gray = gray.copy()
    prev_points = next_points
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

프레임 단계 + 업데이트 단계

1. SIFT

In [28]:
import cv2

In [29]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [30]:
sift = cv2.SIFT_create()

In [31]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    frame = cv2.drawKeypoints(frame, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imshow("SIFT", frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1-1. SIFT 특징점 갯수 제한

In [32]:
import cv2

In [33]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [34]:
sift = cv2.SIFT_create(contrastThreshold=0.02)
max_keypoints = 100  # 원하는 특징점 갯수

In [35]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    if len(keypoints) > max_keypoints:
        keypoints = sorted(keypoints, key=lambda x: -x.response)[:max_keypoints]
        
    frame = cv2.drawKeypoints(frame, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    cv2.imshow("SIFT", frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

2. ORB

In [36]:
import cv2

In [37]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [38]:
orb = cv2.ORB_create()

In [39]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    keypoints = orb.detect(gray, None)
    frame = cv2.drawKeypoints(frame, keypoints, None, color=(0, 255, 0), flags=0)
    
    cv2.imshow("ORB", frame)
    
    if cv2.waitKey(30) & 0xFF ==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

2-1. ORB 불필요한 특징점 및 중복된 특징점 제거

In [40]:
import cv2
import numpy as np

In [41]:
cap = cv2.VideoCapture('slow_traffic_small.mp4')

In [42]:
orb = cv2.ORB_create()
min_keypoint_size = 10
duplicate_threshold=10

In [43]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    keypoints = orb.detect(gray, None)
    
    mask = np.ones(len(keypoints), dtype=bool)
    for i, kp1 in enumerate(keypoints):
        if mask[i]:
            for j, kp2 in enumerate(keypoints[i+1:]):
                if mask[i+j+1] and np.linalg.norm(np.array(kp1.pt) - np.array(kp2.pt)) < duplicate_threshold:
                    mask[i+j+1] = False
                    
    keypoints = [kp for i, kp in enumerate(keypoints) if mask[i]]
    frame = cv2.drawKeypoints(frame, keypoints, None, color=(0, 255, 0), flags=0)
    
    cv2.imshow("ORB", frame)
    
    if cv2.waitKey(30) & 0xFF ==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

특징점 매칭

1. ORB + Brute-Force

In [44]:
import cv2

In [45]:
img1 = cv2.imread('./cat01.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('./cat01.jpg', cv2.IMREAD_GRAYSCALE)

In [46]:
orb = cv2.ORB_create()

In [47]:
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

In [48]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)
matches = sorted(matches, key=lambda x: x.distance)
result = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("Matches", result)

num_matches = len(matches)
num_good_matches = sum(1 for m in matches if m.distance < 50)
matching_percent = (num_good_matches / num_matches) * 100

print("매칭 퍼센트: %.2f%%" % matching_percent)

cv2.waitKey(0)
cv2.destroyAllWindows()

매칭 퍼센트: 100.00%


2. FLANN

In [49]:
import cv2
import numpy as np

In [50]:
img1 = cv2.imread('./cat01.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('./cat01.jpg', cv2.IMREAD_GRAYSCALE)

In [51]:
orb = cv2.ORB_create()

In [52]:
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

In [53]:
index_params = dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

실시간 객체 추적 시스템 구현

In [54]:
import cv2

cap = cv2.VideoCapture('slow_traffic_small.mp4')

sift = cv2.SIFT_create()

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prev_keypoints, prev_descriptors = sift.detectAndCompute(prev_gray, None)

bbox = cv2.selectROI('Select Object', prev_frame, False, False)

x, y, w, h = bbox
track_window = (x, y, w, h)

roi = prev_gray[y:y+h, x:x+w]
roi_keypoints, roi_descriptors = sift.detectAndCompute(roi, None)
matcher = cv2.BFMatcher(cv2.NORM_L2)
matches = matcher.match(prev_descriptors, roi_descriptors)
matches = sorted(matches, key=lambda x: x.distance)
matching_indices = [m.trainIdx for m in matches]

color = (0, 255, 0)

while True:
    ret, frame = cap.read()
    
    if not ret:
            break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_keypoints, roi_descriptors = sift.detectAndCompute(roi_gray, None)
    matches = matcher.match(prev_descriptors, roi_descriptors)
    matches = sorted(matches, key=lambda x: x.distance)

    for match in matches:
        pt1 = prev_keypoints[match.queryIdx].pt
        pt2 = roi_keypoints[match.trainIdx].pt
        x1, y1 = map(int, pt1)
        x2, y2 = map(int, pt2)
        cv2.circle(frame, (x+x2, y+y2), 3, color, -1)

    cv2.imshow('Object Tracking', frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    prev_gray = gray.copy()
    prev_keypoints = roi_keypoints
    prev_descriptors = roi_descriptors

cap.release()
cv2.destroyAllWindows()